<a href="https://colab.research.google.com/github/martinpius/PYTORCH/blob/main/Simple_RNN_with_LSTM_architecture_using_pytorch_.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount("/content/drive", force_remount = True)
try:
  COLAB = True
  import torch
  print(f"You are on Google CoLaB with pytorch version: {torch.__version__}")
except Exception as e:
  print(f"{type(e)}: {e}\n>>>please load your google drive....")
  COLAB = False
#Assigning the GPU device when available:
if torch.cuda.is_available():
  device = torch.device('cuda')
else:
  device = torch.device('cpu')
def time_fmt(t:float = 123.187)->float:
  h = int(t / (60 * 60))
  m = int(t % (60 * 60) / 60)
  s = int(t % 60)
  return f"{h}: {m:>02}: {s:>05.2f}"
print(f">>>time testing\tplease wait...\n>>>time elapse: \t{time_fmt()}")

Mounted at /content/drive
You are on Google CoLaB with pytorch version: 1.8.1+cu101
>>>time testing	please wait...
>>>time elapse: 	0: 02: 03.00


In [2]:
#In this notbook we are going to train a simple RNN with an LSTM architecture (many-one) using
#MNIST images as sequential data. MNIST images has the shape of 1, 28, 28 thus we may treat
#28 pixels as tie steps and the next 28 pixels as feature's size for each sample. We will 
#squeeze the channel dimension to have the shape of (batch, sequence_length,input_size) = say (64, 28,28)

In [3]:
#We first import the necessary packages from torch and other libraries:

In [4]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
import torchvision.datasets as datasets
from torch.utils.data import DataLoader
from torchvision.transforms import transforms
import numpy as np
import time, datetime, os, sys
from tqdm import tqdm


In [5]:
#Hyperparameters to be used 
batch_size = 64
epochs = 10
learning_rate = 0.001
input_size = 28
sequence_length = 28
num_layers = 2
num_classes = 10
hidden_size = 256


In [6]:
#We now defining our simple rnn with LSTM architecture using the following class:
class RNN_LSTM(nn.Module):
  def __init__(self, input_size, hidden_size, num_layers, num_classes):
    super(RNN_LSTM, self).__init__()
    self.hidden_size = hidden_size
    self.num_layers = num_layers
    self.lstm = nn.LSTM(input_size, hidden_size, num_layers, batch_first = True)
    self.fc = nn.Linear(hidden_size * sequence_length, num_classes)
  
  def forward(self, x):
    #initialize hidden and cell states:
    h0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
    c0 = torch.zeros(self.num_layers, x.size(0), self.hidden_size).to(device)
    out,_ = self.lstm(x, (h0, c0))
    out = out.reshape(out.shape[0], -1)
    out = self.fc(out)
    return out

In [7]:
#Instantiating the model class:
model = RNN_LSTM(input_size, hidden_size, num_layers,num_classes).to(device)

In [8]:
#Loading data from torch:
train_data = datasets.MNIST(root = 'train_dataset/', train = True, transform = transforms.ToTensor(), download = True)
train_loader = DataLoader(dataset = train_data, batch_size = batch_size, shuffle = True)
test_data = datasets.MNIST(root = 'test_dataset/', train = False, transform = transforms.ToTensor(), download = True)
test_loader = DataLoader(dataset = test_data, batch_size = batch_size, shuffle = True)
x_train_batch, y_train_batch = next(iter(train_loader))
print(f"train_batch_x_shape: {x_train_batch.shape}\ttrain_batch_y_shape: {y_train_batch.shape}")


Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting train_dataset/MNIST/raw/train-images-idx3-ubyte.gz to train_dataset/MNIST/raw




Extracting train_dataset/MNIST/raw/train-labels-idx1-ubyte.gz to train_dataset/MNIST/raw




Extracting train_dataset/MNIST/raw/t10k-images-idx3-ubyte.gz to train_dataset/MNIST/raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting train_dataset/MNIST/raw/t10k-labels-idx1-ubyte.gz to train_dataset/MNIST/raw

Processing...
Done!


...

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting test_dataset/MNIST/raw/train-images-idx3-ubyte.gz to test_dataset/MNIST/raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting test_dataset/MNIST/raw/train-labels-idx1-ubyte.gz to test_dataset/MNIST/raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting test_dataset/MNIST/raw/t10k-images-idx3-ubyte.gz to test_dataset/MNIST/raw

Failed to download (trying next):
HTTP Error 503: Service Unavailable




Extracting test_dataset/MNIST/raw/t10k-labels-idx1-ubyte.gz to test_dataset/MNIST/raw

Processing...
Done!
train_batch_x_shape: torch.Size([64, 1, 28, 28])	train_batch_y_shape: torch.Size([64])


In [13]:
#Getting the loss and optimizers
loss_obj = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = learning_rate)


In [14]:
#training loop:
tic = time.time()
for epoch in range(epochs):
  print(f"train starts for epoch: {epoch + 1}:\n>>>>please wait while model is training....")
  for batch_idx,(data, target) in enumerate(tqdm(train_loader)):
    #feed data into GPU if available:
    data = data.to(device = device).squeeze(1)
    target = target.to(device = device)
    #foward pass 
    preds = model(data)
    loss = loss_obj(preds, target)
    #backward pass
    optimizer.zero_grad()
    loss.backward()
    #gradient descent with Adam optimizer
    optimizer.step()
#Monitoring training and validation accuracy:
def _model_checking_(loader, model):
  if loader.dataset.train:
    print(f"\n>>>Please wait while checking metrics for training data:")
  else:
    print(f"\n>>>Please wait while checking metrics for validation data")
  num_correct = 0
  num_examples = 0
  model.eval()
  with torch.no_grad():
    for x, y in loader:
      x = x.to(device = device).squeeze(1)
      y = y.to(device = device)
      preds = model(x)
      _, predictions = preds.max(1)
      num_correct+= (predictions == y).sum()
      num_examples+=predictions.size(0)
  model.train()
  return (num_correct/num_examples)
print(f"the accuracy for the training data is {float(_model_checking_(train_loader, model))*100:.2f}")
print(f"the accuracy for the test data is {float(_model_checking_(test_loader, model))*100:.2f}")
toc = time.time()
print(f"\ntime elapse for training and evaluation is :\t{time_fmt(toc - tic)}")

  1%|          | 6/938 [00:00<00:17, 52.19it/s]

train starts for epoch: 1:
>>>>please wait while model is training....


  1%|          | 8/938 [00:00<00:13, 70.60it/s]

train starts for epoch: 2:
>>>>please wait while model is training....


  1%|          | 8/938 [00:00<00:12, 71.65it/s]

...

train starts for epoch: 9:
>>>>please wait while model is training....


  1%|          | 7/938 [00:00<00:13, 69.89it/s]

train starts for epoch: 10:
>>>>please wait while model is training....


100%|██████████| 938/938 [00:13<00:00, 72.12it/s]



>>>Please wait while checking metrics for training data:
the accuracy for the training data is 99.48

>>>Please wait while checking metrics for validation data
the accuracy for the test data is 98.79

time elapse for training and evaluation is :	0: 02: 19.00
